<a href="https://colab.research.google.com/github/Zencriel/Bias_In_MachineLearning_Research/blob/main/Experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv("train.csv")
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [ ]:
train = data.dropna(how = 'any', axis = 0)

In [ ]:

train = train.drop('Loan_ID', axis=1)


In [ ]:
list(train.columns)

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History',
 'Property_Area',
 'Loan_Status']

In [ ]:
data['Gender'].value_counts()

,count
Gender,
Male,489
Female,112


In [ ]:
data['Loan_Status'].value_counts()

,count
Loan_Status,
Y,422
N,192


In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop('Loan_Status', axis=1)
y = train['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train.shape

(384, 11)

In [ ]:
X_test.shape

(96, 11)

In [ ]:
y_train.shape

(384,)

In [ ]:
y_test.shape

(96,)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns

le = LabelEncoder()

for feature in categorical_features:
    X_train[feature] = le.fit_transform(X_train[feature])
    X_test[feature] = le.transform(X_test[feature])

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                          cv=5, n_jobs=-1, scoring='accuracy')

grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)

best_rf_classifier = RandomForestClassifier(**grid_search.best_params_, random_state=42)
best_rf_classifier.fit(X_train, y_train)


Best Hyperparameters: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}


RandomForestClassifier(max_depth=5, min_samples_leaf=2, n_estimators=50,
                       random_state=42)

In [ ]:
import pickle

with open('my_model.pkl', 'wb') as f:
    pickle.dump(model, f)

### **Testing Below**

In [ ]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = best_rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.8229166666666666
Precision: 0.8
Recall: 1.0
F1 Score: 0.8888888888888888


### **NAIVE BAYES BELOW**

In [ ]:

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

nb_classifier = GaussianNB()

grid_search_nb = GridSearchCV(estimator=nb_classifier, param_grid=param_grid_nb,
                             cv=5, n_jobs=-1, scoring='accuracy')

grid_search_nb.fit(X_train, y_train)

best_nb_classifier = GaussianNB(**grid_search_nb.best_params_)
best_nb_classifier.fit(X_train, y_train)

y_pred_nb = best_nb_classifier.predict(X_test)


In [ ]:

accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb)
recall_nb = recall_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb)

print(f"Naive Bayes Accuracy: {accuracy_nb}")
print(f"Naive Bayes Precision: {precision_nb}")
print(f"Naive Bayes Recall: {recall_nb}")
print(f"Naive Bayes F1 Score: {f1_nb}")

Naive Bayes Accuracy: 0.8125
Naive Bayes Precision: 0.8048780487804879
Naive Bayes Recall: 0.9705882352941176
Naive Bayes F1 Score: 0.88
